In [2]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """ You are a careful SQLite analyst
Rules:
- think step-by-step
- when you need data, call the tool 'execute_sql' with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

agent = create_agent(model="ollama:qwen3:4b",
                     tools=[execute_sql],
                     system_prompt=SYSTEM_PROMPT,
                     context_schema=RuntimeContext)

c:\Users\sadaw\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
question = "what are different genres?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what are different genres?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (692542a5-70e8-46de-8eed-fc184a592383)
 Call ID: 692542a5-70e8-46de-8eed-fc184a592383
  Args:
    query: SELECT DISTINCT genre_name FROM genres
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: genres
[SQL: SELECT DISTINCT genre_name FROM genres]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (721f04c7-10fe-4a7c-a1ca-2fee55be52ab)
 Call ID: 721f04c7-10fe-4a7c-a1ca-2fee55be52ab
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table'
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (39dde725-e50b-40cf-8909-49c8398cfcd0)
 Call ID: 39dde725-e50b-40cf-8909-49c8398cfcd0
  Args:
    query: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (29693f40-d483-4e1d-9584-4d9e6856c356)
 Call ID: 29693f40-d483-4e1d-9584-4d9e6856c356
  Args:
    query: SELECT name FROM sqlite